In [19]:
import numpy as np
import skimage
import utils
import pathlib

In [20]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
        Otsu's thresholding algorithm that segments an image into 1 or 0 (True or False)
        The function takes in a grayscale image and outputs a threshold value

        args:
            im: np.ndarray of shape (H, W) in the range [0, 255] (dtype=np.uint8)
        return:
            (int) the computed thresholding value
    """
    assert im.dtype == np.uint8
    ### START YOUR CODE HERE ### (You can change anything inside this block) 
    # You can also define other helper functions
    # Compute normalized histogram
    hist, bins = np.histogram(im.flatten(), bins=256, range=[0, 256], density=True)
    
    variance = 0
    threshold = 0

    for t in range(1, 256):
        #cumulative sum
        s1 = np.sum(hist[:t])
        s2 = np.sum(hist[t:])

        #cumulative means
        m1 = np.sum(np.arange(t) * hist[:t]) / (s1 + 1e-10)
        m2 = np.sum(np.arange(t, 256) * hist[t:]) / (s2 + 1e-10)

        # between-class variance
        bc_variance = s1 * s2 * ((m1 - m2) ** 2)

        # update threshold if variance is higher
        if bc_variance > variance:
            variance = bc_variance
            threshold = t

    return threshold
    ### END YOUR CODE HERE ###

In [21]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

Reading image: images/thumbprint.png
Found optimal threshold: 154
Saving image to: image_processed/thumbprint-segmented.png
Reading image: images/rice-shaded.png
Found optimal threshold: 135
Saving image to: image_processed/rice-shaded-segmented.png
